In [11]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from video_block import * 
from inflator import *
from triplet_loss import *

ModuleNotFoundError: No module named 'video_block'

In [1]:
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

/home/adrian/anaconda3/envs/VE/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
class NullModel(Module):
    def __init__(self): pass
    def forward(*x, **kwargs): return x

class NullLoss(Module):
    def __init__(self): pass
    def forward(*x, **kwargs): return torch.tensor(0.,requires_grad=True)
    
def null_splitter(m): return [[],[]]

In [3]:
#export
def read_data(eugenio):
    path_data = '/mnt/data/eugeniomarinelli/videodata/df.csv' if eugenio else '/mnt/data/adrianlopez/Videos/Charades/df.csv'
    df = pd.read_csv(path_data, index_col=0).dropna()
    return df 

In [4]:
df = read_data(False)

In [5]:
#export
def get_datasets(df, l, n_lbl, n_el):
    def get_vid_path(df:pd.Series): return Video(df['paths'].split('\n'))
    def get_lbls(df:pd.Series): return df['lbl']

    tfms = [[get_vid_path, ResizeTime(l=l), Video.create],
            [get_lbls, Categorize()]]

    splits = RandomSplitter(valid_pct=0.2, seed=42)(df)
    splits = [uniformize_dataset(idxs, df.iloc[idxs]['lbl'] , n_lbl=n_lbl, n_el=n_el) for idxs in splits]

    dsets = Datasets(df, tfms, splits=splits)
    return dsets

In [6]:
ds = get_datasets(df, l=20, n_lbl=4, n_el=8)
for i in range(-1,-10,-1): print(ds.valid[i][1])

TensorCategory(86)
TensorCategory(86)
TensorCategory(86)
TensorCategory(86)
TensorCategory(86)
TensorCategory(86)
TensorCategory(86)
TensorCategory(86)
TensorCategory(32)


In [7]:
#export
def get_dls(df, sz, l, n_lbl, n_el):
    dsets = get_datasets(df, l, n_lbl, n_el)
    dls = dsets.dataloaders(bs=n_el*n_lbl,
                            shuffle_train=False,
                            after_item=[Resize(sz), ToTensor()],
                            after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])
    return dls

In [8]:
dls = get_dls(df, 224, 20, 4, 8)
dls.one_batch()

(TensorVideo([[[[[0.8078, 0.7922, 0.7725,  ..., 0.3294, 0.3412, 0.3490],
            [0.8000, 0.8078, 0.8118,  ..., 0.3255, 0.3373, 0.3451],
            [0.8039, 0.8118, 0.7882,  ..., 0.3216, 0.3333, 0.3451],
            ...,
            [0.6980, 0.6706, 0.6863,  ..., 0.1020, 0.1020, 0.1020],
            [0.6941, 0.6745, 0.6745,  ..., 0.0941, 0.0941, 0.0941],
            [0.6824, 0.6706, 0.6745,  ..., 0.0941, 0.0941, 0.0941]],
 
           [[0.7961, 0.8039, 0.8039,  ..., 0.3333, 0.3412, 0.3529],
            [0.8196, 0.8000, 0.8000,  ..., 0.3294, 0.3373, 0.3490],
            [0.7882, 0.8157, 0.8039,  ..., 0.3255, 0.3373, 0.3490],
            ...,
            [0.6863, 0.6784, 0.6745,  ..., 0.1020, 0.1020, 0.1020],
            [0.7020, 0.6667, 0.6667,  ..., 0.0941, 0.0941, 0.0941],
            [0.6863, 0.6627, 0.6627,  ..., 0.0941, 0.0941, 0.0941]],
 
           [[0.8039, 0.7882, 0.7843,  ..., 0.3373, 0.3451, 0.3451],
            [0.7882, 0.8000, 0.8078,  ..., 0.3333, 0.3451, 0.3412],
   

In [9]:
#export
def get_learner(df, l, sz, n_lbl, n_el):
    dls = get_dls(df, sz, l, n_lbl, n_el)
    
    head, body = inflate(create_head(1024, len(dls.vocab), lin_ftrs=[256])), inflate(create_body(resnet34, cut=-2)) 
    learn = Learner(dls, 
    #                 NullModel(),
                    TLModel(body,head),
    #                 loss_func=NullLoss(),
                    MixedLoss(alpha=1.0, margin=.2),
    #                 splitter=null_splitter,
                    splitter=my_splitter,
    #                 opt=Optimizer([0], noop),
    #                 metrics=lambda *x: 0)
                    metrics=tl_accuracy)
    return learn

In [10]:
#exporti
df = read_data(False)
learn = get_learner(df, l=40, sz=224, n_lbl=4, n_el=8)
learn.fine_tune(1)

/home/adrian/anaconda3/envs/VE/lib/python3.7/site-packages/fastai/callback/schedule.py:68: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  idx = (pos >= pcts).nonzero().max()


KeyboardInterrupt: 

In [45]:
with learn.parallel_ctx(device_ids=[0,1]):
    learn.fine_tune(4)

KeyboardInterrupt: 

In [3]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_charades.ipynb.
Converted 05_ufc.ipynb.
Converted 06_distributed_inflated_NN.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.


In [ ]:
# default_exp charades